# What happens:
Files are created, send email with excel script

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell import Cell
from openpyxl.styles import Color, Font, PatternFill
import glob

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/004 Redeployment"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

Please enter the RD (YYYYMMDD): 20221031


### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

## Function

In [4]:
def save_file(file_name, sheet_name, ws_nr, sqls):
    # Create file
    wb = Workbook()
    for index, sheet in enumerate(sheet_name):
        wb.create_sheet(sheet)
        
    std = wb.get_sheet_by_name('Sheet')
    wb.remove_sheet(std)
    wb.save(filename = file_name)
    
    dfs = []
    # Select SQLs
    for index, sheet in enumerate(sheet_name):
        try:
            connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                            reporting_date = rd, partition = ws_nr[index]))
            df_temp= pd.read_sql_query( sqls[index], connection_RAY)
            dfs.append(df_temp)
        except:
            dfs.append(pd.DataFrame())
            print( str(ws_nr[index]) + " is not created")
            
    writer = pd.ExcelWriter(file_name)
    for index, df in enumerate(dfs):
        try:
            df.to_excel(writer, index=False, sheet_name = sheet_name[index])
        except:
            print()
    
    writer.save()
    print('DataFrame is written to Excel File successfully.')


### Function 2( Cons & Uncons)

In [5]:
def modify_cons_uncons(file_name):
    wb2 = load_workbook(file_name)
    wb2.create_sheet('Überblick')
    
    #2
    connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                            reporting_date = rd, partition = 2))
    df = pd.read_sql_query( """select reporting_Date, partition_key, booking_company, 'RBI_GRP' as consolidation_level, bs_indicator, sum(rwa) as rwa	
from rzb_Cdr	
where record_source = 'T_CDR'	
and booking_company = 'RBI'	
group by reporting_Date, partition_key, booking_company, bs_indicator	""", connection_RAY)

    df.loc[df.shape[0]] = [None, None, None, None, None, None]
    
    #194
    connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                            reporting_date = rd, partition = 194))
    df_temp= pd.read_sql_query( """select reporting_Date, partition_key, booking_company, 'SOLO' as consolidation_level, bs_indicator, sum(rwa) as rwa	
from rzb_Cdr	
where record_source = 'T_CDR'	
and booking_company = 'RBI'	
group by reporting_Date, partition_key, booking_company, bs_indicator	""", connection_RAY)
    df = df.append(df_temp)
    sql1 = wb2['Überblick']
    for idx_c, header in enumerate(df.columns.values):
        sql1.cell(row=1, column=idx_c+1).value = header
    
    for idx, row in enumerate(df.itertuples(index = True)):
        for idx_c, header in enumerate(df.columns.values):
            sql1.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
    wb2.save(file_name)
    print('DataFrame is written to Excel File successfully.')

### Function 3( CVA)

In [6]:
def save_file_cva(file_name, sheet_name, ws_nr, sqls):
    dfs = []
    # Select SQLs
    for index, sheet in enumerate(sheet_name):
        try:
            connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                            reporting_date = rd, partition = ws_nr[index]))
            df_temp= pd.read_sql_query( sqls[index], connection_RAY)
            dfs.append(df_temp)
        except:
            dfs.append(pd.DataFrame())
            print( str(ws_nr[index]) + " is not created")


    units = ["ALEASS", "AVAL", "CENTRO", "RBCZ", "RBHR", "RBHU", "RBI", "RBRU", "RLBANK", "RLCZ", "TBSK"]
    units_info = ["[ALEASS],[LD_IT],[RBI_GRP],[RLATIT_GRP],[RLGMBH_GRP],[SEG_GCM]", "[AVAL],[LD_UA],[RBI_GRP],[SEG_EE]", "[CENTRO],[RBI_GRP]", "[LD_CZ],[RBCZ],[RBCZ_GRP],[RBI_GRP],[SEG_CE]", "[LD_HR],[RBHR],[RBHR_GRP],[RBI_GRP],[SEG_SEE]", "[LD_HU],[RBHU],[RBHU_GRP],[RBI_GRP],[SEG_CE]", "[RBIAG],[RBI],[RBI_GRP]", "[LD_RU],[RBI_GRP],[RBRU],[SEG_EE]", "[LD_AT],[RBI_GRP],[RLBANK],[RLGMBH_GRP],[SEG_GCM]", "[LD_CZ],[RBCZ],[RBCZ_GRP],[RBI_GRP],[SEG_CE]", "[LD_SK],[RBI_GRP],[SEG_CE],[TBSK],[TBSK_GRP]"]
                
        
    for i, unit in enumerate(units):
        wb = Workbook()
        ws = wb.active
        newdf = dfs[0][(dfs[0].conso_perimeter == units_info[i])]
        for idx_c, header in enumerate(newdf.columns.values):
            ws.cell(row=1, column=idx_c+1).value = header
        
        for idx, row in enumerate(newdf.itertuples(index = True)):
            for idx_c, header in enumerate(newdf.columns.values):
                ws.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        wb.save(filename = unit + "_CVA_" + rd[4:] + ".xlsx")
    print('DataFrame is written to Excel File successfully.')


### OTC Netting

In [7]:
file_name = rd + '_OTC_NETTING.xlsx'
sheet_name = [rd + " RBIAG", rd + " RBI" ]
ws_nr = [44, 46]
sqls = ["""select c.book_code, c.contract_type, c.netting_agreement_ref as netting_agreement, 
    c.bs_type, c.nominal, c.fair_value, c.add_on, c.counterparty as Cocunut, et.entity_type_desc as Counterparty_Type,
    e.entity_desc as Counterparty_Name, c.internal_rating as Internal_Rating, c.attribute_5 as Deal_ID, c.start_date,
    c.maturity_date, c.table_name as Product, c.ead_pre_ccf as EAD_group_by_agreement_pre_net,
    c.ead_netting as EAD_covered_by_netting, c.ead_fi as EAD_covered_by_Financial_Col, 
    c.ead_eff as uncovered_EAD_per_agreement, c.rwa as RWA_per_agreement, t1.collateral_contract_type, 
    t1.attribute_5 as Collateral_ID
from csr c
left join entity e on c.counterparty = e.entity_code
left join entity_type et on e.et_code_user = et.entity_type
left join (select t.contract_reference, 
        sum(t.add_on) as add_on_1,
        t.collateral_reference, 
        t.collateral_contract_type, 
        t.tranche_collateral_type,
        cl.attribute_5,
        sum(t.ead_pre_ccf) as ead_pre_ccf
    from t_cdr t
    left join collateral cl on t.collateral_reference = cl.contract_reference
    where t.collateral_contract_type = 'CSH' and t.tranche_collateral_type = 'COLLATERALIZED'
    group by t.contract_reference, t.collateral_reference, t.collateral_contract_type,
    t.tranche_collateral_type, cl.attribute_5
    ) t1 on c.contract_reference = t1.contract_reference
where ((c.bs_type in ('DAF','DAO','DLF','DLO')
or c.table_name like 'CREDI%'))
and c.contract_type <> 'DRM'""", """select c.book_code, c.contract_type, c.netting_agreement_ref as netting_agreement, 
    c.bs_type,
    c.nominal,
    c.fair_value,
    c.add_on,
    c.counterparty as Cocunut, 
    et.entity_type_desc as Counterparty_Type, 
    e.entity_desc as Counterparty_Name, 
    c.internal_rating as Internal_Rating, 
    c.attribute_5 as Deal_ID, 
    case when c.table_name = 'SWAP' then s.value_date 
     when c.table_name = 'CAPFLOOR' then cf.value_date 
         else c.start_date
---      when c.table_name = 'OPTIONS' then o.value_date 
    end start_date,
    c.maturity_date,
    c.table_name as Product,
    c.ead_pre_ccf as EAD_group_by_agreement_pre_net, 
    c.ead_netting as EAD_covered_by_netting, 
    c.ead_fi as EAD_covered_by_Financial_Col, 
    c.ead_eff as uncovered_EAD_per_agreement, 
    c.rwa as RWA_per_agreement,
    t1.collateral_contract_type,
    t1.attribute_5 as Collateral_ID
from csr c
left join entity e on c.counterparty = e.entity_code
left join entity_type et on e.et_code_user = et.entity_type
left join swap s on s.contract_reference = c.contract_reference
   ---left join options o on o.contract_reference = c.contract_reference
left join capfloor cf on cf.contract_reference = c.contract_reference
left join (select t.contract_reference, 
        sum(t.add_on) as add_on_1,
        t.collateral_reference, 
        t.collateral_contract_type, 
        t.tranche_collateral_type,
        cl.attribute_5,
        sum(t.ead_pre_ccf) as ead_pre_ccf
    from t_cdr t
    left join collateral cl on t.collateral_reference = cl.contract_reference
    where t.collateral_contract_type = 'CSH' and t.tranche_collateral_type = 'COLLATERALIZED'
    group by t.contract_reference, t.collateral_reference,
    t.collateral_contract_type, t.tranche_collateral_type, cl.attribute_5
    ) t1 on c.contract_reference = t1.contract_reference
where ((c.bs_type in ('DAF','DAO','DLF','DLO')
or c.table_name like 'CREDI%'))
and c.contract_type <> 'DRM'"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### RBSG

In [8]:
file_name = rd + '_RBSG.xlsx'
sheet_name = [rd + " RBSG_CONS", rd + " RBSG&HO_CONS", rd + "RBSG&HO_UNCONS" ]
ws_nr = [46, 46, 194]
sqls = ["""select v.partition_key, v.reporting_date, v.table_name, v.contract_reference, v.attribute_5,	
v.facility_reference, v.start_date, v.maturity_date, v.tenor_original, v.tenor_residual,	
v.residual_maturity, v.bs_indicator, v.counterparty_code, v.counterparty_name, v.industry_code_1,	
v.industry_code_2, v.nace_code, v.profit_center, v.account_manager, v.internal_rating,	
v.rating_model, v.master_rating_long, v.master_rating_short, v.rating_agency, v.rating_code,	
v.counterparty_type, v.counterparty_category, v.counterparty_type_desc, v.default_flag,	
v.gcc_head_code, v.gcc_head_name, v.gcc_head_rating, v.risk_country, v.risk_country_name,	
v.risk_region, v.residence_country, v.residence_country_name, v.residence_region, v.product_group,	
v.contract_type, v.contract_type_desc, v.seniority, v.counterparty_new_code, v.counterparty_new_name,	
v.gcc_head_new_code, v.gcc_head_new_name, v.risk_country_new, v.risk_country_new_name, v.collateral_type,	
v.booking_company, v.book_code, v.booking_country, v.currency, Null as "gross_exposure", 	
round(v.ead_pre_ccf,10) ead_pre_ccf, round(v.ead,10) ead, round(v.ead_pre_ccf_secured,10) as ead_pre_ccf_secured,	
round(v.ead_pre_ccf_unsecured,10) as ead_pre_ccf_unsecured, v.rw_eff as rw_eff, round(v.rwa,10) as rwa,	
round(v.deductions,10) as deductions, v.pd as pd, v.pd_used as pd_used, round(v.lgd,10) as lgd,	
round(v.el_estimate,10) as el_estimate, round(v.el_regulatory,10) as el_regulatory,	
round(v.provisions,10) as provisions, v.securitized, v.record_source, c.exch_rate as exch_rate,	
round((v.ead_pre_ccf/c.exch_rate),10) as ead_pre_orig_fx, 	
round((v.ead_pre_ccf_secured/c.exch_rate),10) as secured_part_orig_fx,	
round((v.ead_pre_ccf_unsecured/c.exch_rate),10) as unsecured_part_orig_fx,	
round((v.rwa/c.exch_rate),10) as rwa_orig_fx, round((v.ec_credit),10) as economic_capital,	
round(wcv,10),round(wcv_guar,10), V.ccf as ccf,	
case when v.contract_type = 'DL0213' then 'ASS'	
     else v.bs_type end bs_type_balance_sheet,	
add_on, fair_value	
from rzb_cdr v	
left join currency c on c.ccy_code = decode(record_source,'WCV','EUR',v.currency)	
where book_code in ('RBSG_B','RBSG_T') ---and v.partition_key = 'JJJJMMTT0048' 	
---and c.partition_key = 'JJJJMMTT0000'""", """select v.partition_key, v.reporting_date, v.table_name, v.contract_reference, v.attribute_5, v.facility_reference,
v.start_date, v.maturity_date, v.tenor_original, v.tenor_residual, v.residual_maturity, v.bs_indicator,
v.counterparty_code, v.counterparty_name, v.industry_code_1, v.industry_code_2, v.nace_code, v.profit_center,
v.account_manager, v.internal_rating, v.rating_model, v.master_rating_long, v.master_rating_short, v.rating_agency,
v.rating_code, v.counterparty_type, v.counterparty_category, v.counterparty_type_desc, v.default_flag,
v.gcc_head_code, v.gcc_head_name, v.gcc_head_rating, v.risk_country, v.risk_country_name, v.risk_region,
v.residence_country, v.residence_country_name, v.residence_region, v.product_group, v.contract_type,
v.contract_type_desc, v.seniority, v.counterparty_new_code, v.counterparty_new_name, v.gcc_head_new_code,
v.gcc_head_new_name, v.risk_country_new, v.risk_country_new_name, v.collateral_type, v.booking_company,
v.book_code, v.booking_country, v.currency, Null as "gross_exposure", round(v.ead_pre_ccf,10) ead_pre_ccf,
round(v.ead,10) ead, round(v.ead_pre_ccf_secured,10) as ead_pre_ccf_secured,
round(v.ead_pre_ccf_unsecured,10) as ead_pre_ccf_unsecured,
v.rw_eff as rw_eff, round(v.rwa,10) as rwa, round(v.deductions,10) as deductions, v.pd as pd, v.pd_used as pd_used,
round(v.lgd,10) as lgd, round(v.el_estimate,10) as el_estimate, round(v.el_regulatory,10) as el_regulatory,
round(v.provisions,10) as provisions, v.securitized, v.record_source, c.exch_rate as exch_rate,
round((v.ead_pre_ccf/c.exch_rate),10) as ead_pre_orig_fx,
round((v.ead_pre_ccf_secured/c.exch_rate),10) as secured_part_orig_fx,
round((v.ead_pre_ccf_unsecured/c.exch_rate),10) as unsecured_part_orig_fx,
round((v.rwa/c.exch_rate),10) as rwa_orig_fx,
round((v.ec_credit),10) as economic_capital, round(wcv,10),round(wcv_guar,10), V.ccf as ccf,
case when v.contract_type = 'DL0213' then 'ASS'
     else v.bs_type end bs_type_balance_sheet
from rzb_cdr v
left join currency c on c.ccy_code = decode(record_source,'WCV','EUR',v.currency)
where operating_subsegment = 'Functional Cooperation'
and operating_Segment = 'Non-Core'""", """select v.partition_key, v.reporting_date, v.table_name, v.contract_reference, v.attribute_5, v.facility_reference,	
v.start_date, v.maturity_date, v.tenor_original, v.tenor_residual, v.residual_maturity, v.bs_indicator,	
v.counterparty_code, v.counterparty_name, v.industry_code_1, v.industry_code_2, v.nace_code, v.profit_center,	
v.account_manager, v.internal_rating, v.rating_model, v.master_rating_long, v.master_rating_short, v.rating_agency,	
v.rating_code, v.counterparty_type, v.counterparty_category, v.counterparty_type_desc, v.default_flag,	
v.gcc_head_code, v.gcc_head_name, v.gcc_head_rating, v.risk_country, v.risk_country_name, v.risk_region, 	
v.residence_country, v.residence_country_name, v.residence_region, v.product_group, v.contract_type, v.contract_type_desc, v.seniority,	
v.counterparty_new_code, v.counterparty_new_name, v.gcc_head_new_code, v.gcc_head_new_name, v.risk_country_new,	
v.risk_country_new_name, v.collateral_type, v.booking_company, v.book_code, v.booking_country, v.currency, 	
Null as "gross_exposure", round(v.ead_pre_ccf,10) as ead_pre_ccf, round(v.ead,10) as ead,	
round(v.ead_pre_ccf_secured,10) as ead_pre_ccf_secured,	
round(v.ead_pre_ccf_unsecured,10) as ead_pre_ccf_unsecured, v.rw_eff as rw_eff, round(v.rwa,10) as rwa,	
round(v.deductions,10) as deductions, v.pd as pd, v.pd_used as pd_used, round(v.lgd,10) as lgd,	
round(v.el_estimate,10) as el_estimate, 	
round(v.el_regulatory,10) as el_regulatory, round(v.provisions,10) as provisions,	
v.securitized as securitized, v.record_source, c.exch_rate as exch_rate,	
round((v.ead_pre_ccf/c.exch_rate),10) as ead_pre_orig_fx,	
round((v.ead_pre_ccf_secured/c.exch_rate),10) as secured_part_orig_fx,	
round((v.ead_pre_ccf_unsecured/c.exch_rate),10) as unsecured_part_orig_fx,	
round((v.rwa/c.exch_rate),10) as rwa_orig_fx,	
round((v.ec_credit),10) as economic_capital,	
round(wcv,10),round(wcv_guar,10),	
V.CCF as ccf,	
case when v.contract_type = 'DL0213' then 'ASS'	
     else v.bs_type end bs_type_balance_sheet	
from rzb_cdr v	
left join currency c on c.ccy_code = decode(record_source,'WCV','EUR',v.currency)	
where operating_subsegment = 'Functional Cooperation'	
and operating_Segment = 'Non-Core'	"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### RLCZ & RLRE

In [9]:
file_name = rd + '_RLCZ_RLRE.xlsx'
sheet_name = [rd + "  RLCZ_GRP_FMA_STD", rd + " ALL_IRB", rd + "UNCONS" ]
ws_nr = [171, 192, 194]
sqls = ["""select * from rzb_cdr where booking_company in ('RLRE','RLREELEN','RLREJOTA','RLRETE','RLRESR','RLCZ')	""",
       """select * from rzb_cdr where booking_company in ('RLRE','RLREELEN','RLREJOTA','RLRETE','RLRESR','RLCZ')	""",
       """select * from rzb_cdr where booking_company in ('RLRE','RLREELEN','RLREJOTA','RLRETE','RLRESR','RLCZ')	"""]
save_file(file_name, sheet_name, ws_nr, sqls)

171 is not created


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### RBIAG

In [10]:
file_name = rd + '_RBIAG.xlsx'
sheet_name = [rd + " RBIAG"]
ws_nr = [44]
sqls = ["""select t.partition_key, t.book_code,t.table_name,t.contract_type,ct.ct_desc_user,round(sum(t.ead),10) as ead,round(sum(t.ead_pre_ccf),10) as ead_pre_ccf, round(sum(t.rwa),10) as rwa, round(sum(t.rwa*0.08),10) as rgc, round(sum(net_ead),10) as net_ead,	
round(sum(ec_credit),10) as ec_credit	
from rzb_cdr t	
left join contract_types ct on t.contract_type = ct.ct_code_user	
where t.ead > 0 ---and t.partition_key = 'JJJJMMTT0044' and ct.partition_key = 'JJJJMMTT0000'	
group by t.partition_key, t.book_code,t.table_name,t.contract_type,ct.ct_desc_user	"""]

save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### RBIAG Global treasury

In [11]:
file_name = rd + '_RBIAG_GLOBAL_TREASURY.xlsx'
sheet_name = [rd + " RBIAG TREASURY"]
ws_nr = [44]
sqls = ["""  select * from 	
  (	
  select 	
    case	
    when contract_type like 'TM%' 	
    then 'global treasury'	
    when pc_code  like '6%' 	
        or pc_code   like '7%' 	
        or pc_code   like '8%' 	
        or pc_code   like '9%' 	
    then 'global treasury'	
    /* corporate sales - kalinka  */	
    when (pc_code   like'01*%' or pc_code   like'02*%')	
        and bs_type in ('DAO','DAF','DLO','DLF','SPT')	
    then 'global treasury'	
    /* financial sales rettl */	
    when pc_code   like'11*%'	
        and bs_type in ('DAO','DAF','DLO','DLF','SPT')	
    then 'global treasury'	
    when table_name = 'REPO'	
    then 'global treasury'	
    else pc_code  	
    end unit,	
    rzb_cdr.* 	
    from rzb_cdr	
    where booking_company in ('RBI') and       	
    /*exclude GEBOS TM0205 */	
            (	
             (contract_type like 'TM%' and attribute_5 not like '%GEBOS%') or	
             (contract_type not like 'TM%')	
             )	
    and (contract_type not like 'PA0401' or contract_type is NULL)	
    and NVL(record_source,0) not like 'WCV'  	
------- Ausschliessen RZB Zukunftsvorsorge pc code 81	
------- Zukunftsvorsorge Artsdeals kommen derzeit nicht in rzb_cdr da alle ead 0 0 haben	
------  haben kein Profitcenter , in t_cdr aber ueber attribute_7 = '***ZUV' zu erkennen	
------  sind auch alle in Nettingagreements , eventuell ueber diese erkennbar	
  and nvl(pc_code,0) <> '81'	
    and (bs_type not in ('VIR','TRA') or bs_type is NULL)  	
    and (securitized not like 'T' or securitized is NULL)	
    --and partition_key = 'JJJJMMTT0048'	
  ) 	
where not (unit is null and table_name <> 'REPO_NETTING_VOL')	"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### REPO

In [12]:
file_name = rd + '_REPO.xlsx'
sheet_name = [rd + " REPO Data", "Triparty"]
ws_nr = [46, 46]
sqls = ["""WITH        
 combine_repo       
AS (        
 select sub01.partition_key, sub01.counterparty_original as Repo_Ctpy, e1.entity_desc as Repo_Ctpy_Name,       
 e1.country_code as Repo_Ctpy_Risk_Country, e1.et_code_user as Repo_Ctpy_type, 
 sub01.internal_rating_original as Counterparty_Rating, sub01.netting_agreement_reference,       
 sub01.table_name, sub01.contract_type, ct.ct_desc_user, sub01.dealbook, sub01.booking_company,
 sub01.exposure_attribute_5, r.contract_reference, r.repo_type, s.security_reference,
 s.security_type, e2.entity_code as Issuer, e2.entity_desc as Issuer_Name, e2.country_code as Issuer_Risk_Country,       
 e2.et_code_user as Issuer_type, et2.entity_type_desc as Issuer_Type_Desc,
 case
  when sub01.table_name='REPO_NETTING_VOL' then sub01.EAD_PRE_CCF_Agg
  when r.repo_type = 'R' then rp.securities_market_value*c2.exch_rate
  when r.repo_type = 'RR' then r.cash_nominal*c1.exch_rate
  else Null
 end as exposure_in_EUR,
 case  when r.repo_type = 'R' then r.cash_nominal*c1.exch_rate
  when r.repo_type = 'RR' then rp.securities_market_value*c2.exch_rate
  else Null 
 end as collateral_in_EUR, sub01.Collateralised, sub01.Uncollateralised, sub01.EAD_PRE_CCF_Agg, sub01.RWA_Agg
 from      
  (select t.partition_key, t.counterparty_original, t.netting_agreement_reference,  t.dealbook,  t.booking_company,
  t.contract_type,  t.table_name,  t.internal_rating_original,  t.exposure_attribute_5,  t.contract_Reference, 
  sum(case when t.tranche_collateral_type like 'COLL%' then t.ead_pre_ccf end) as Collateralised, 
  sum(case when t.tranche_collateral_type not like 'COLL%' then t.ead_pre_ccf 
   end) as Uncollateralised, 
  sum(t.ead_pre_ccf) as EAD_PRE_CCF_Agg, 
  sum(t.rwa) as RWA_Agg 
  from t_cdr t     
  group by t.partition_key, t.contract_reference, t.booking_company, t.contract_type, t.exposure_attribute_5,
  t.internal_rating_original,   t.dealbook, t.counterparty_original, t.netting_agreement_reference, 
  t.table_name 
  ) sub01 
 left join repo r on sub01.contract_reference = r.contract_reference 
 left join currency c1 on r.cash_currency = c1.ccy_code     
 left join repo_portfolio rp on r.contract_reference = rp.repo_reference   
 left join currency c2 on rp.ccy_code = c2.ccy_code      
 left join contract_types ct on sub01.contract_type = ct.ct_code_user      
 left join entity e1 on sub01.counterparty_original = e1.entity_code      
 left join security s on rp.security_reference = s.security_reference       
 left join entity e2 on s.issuer_code = e2.entity_code      
 left join entity_type et2 on e2.et_code_user = et2.entity_type      
 where sub01.table_name like 'REP%'       
 and sub01.dealbook in ('HO_B','HO_T','RZBA_B')       
 UNION      
 select sub01.partition_key, sub01.counterparty_original as Repo_Ctpy, e1.entity_desc as Repo_Ctpy_Name,      
 e1.country_code as Repo_Ctpy_Risk_Country, e1.et_code_user as Repo_Ctpy_type,     
 sub01.internal_rating_original as Counterparty_Rating, sub01.netting_agreement_reference,     
 sub01.table_name, sub01.contract_type, ct.ct_desc_user, sub01.dealbook, sub01.booking_company,  
 sub01.exposure_attribute_5, r.contract_reference, r.repo_type, 
 case when r.repo_type = 'R' then s2.security_reference 
 when r.repo_type = 'RR' then s3.security_reference end as security_reference, 
 case when r.repo_type = 'R' then s2.security_type      
 when r.repo_type = 'RR' then s3.security_type end as security_type, 
 case when r.repo_type = 'R' then e2.entity_code      
 when r.repo_type = 'RR' then e3.entity_code end as Issuer, 
 case when r.repo_type = 'R' then e2.entity_desc      
 when r.repo_type = 'RR' then e3.entity_desc end as Issuer_Name,  
 case when r.repo_type = 'R' then e2.country_code       
 when r.repo_type = 'RR' then e3.country_code end as Issuer_Risk_Country,      
 case when r.repo_type = 'R' then e2.et_code_user      
 when r.repo_type = 'RR' then e3.et_code_user end as Issuer_type,     
 case when r.repo_type = 'R' then et2.entity_type_desc      
 when r.repo_type = 'RR' then et3.entity_type_desc end as Issuer_Type_Desc,       
 case      
  when r.repo_type = 'R' then rp.securities_market_value*c2.exch_rate      
  when r.repo_type = 'RR' then r.cash_nominal*c1.exch_rate      
  else Null     
 end as exposure_in_EUR,    
 case  when r.repo_type = 'R' then c.market_value*c3.exch_rate     
  when r.repo_type = 'RR' then c.market_value*c3.exch_rate      
  else Null      
 end as collateral_in_EUR,       
 sub01.Collateralised,      
 sub01.Uncollateralised,       
 sub01.EAD_PRE_CCF_Agg,       
 sub01.RWA_Agg    
 from    
  (select t.partition_key, t.counterparty_original, t.netting_agreement_reference, t.dealbook,     
  t.booking_company, t.contract_type, t.internal_rating_original, t.table_name,       
  t.exposure_attribute_5, t.contract_Reference,      
  sum(case when t.tranche_collateral_type like 'COLL%' then t.ead_pre_ccf     
   end) as Collateralised,    
  sum(case when t.tranche_collateral_type not like 'COLL%' then t.ead_pre_ccf     
   end) as Uncollateralised,     
  sum(t.ead_pre_ccf) as EAD_PRE_CCF_Agg,      
  sum(t.rwa) as RWA_Agg      
  from t_cdr t     
  group by  
  t.partition_key, 
  t.contract_reference,      
  t.contract_type,      
  t.exposure_attribute_5,     
  t.internal_rating_original,      
  t.dealbook,     
  t.booking_company,      
  t.counterparty_original,       
  t.netting_agreement_reference,     
  t.table_name      
  ) sub01    
 left join repo r on sub01.contract_reference = r.contract_reference      
 left join currency c1 on r.cash_currency = c1.ccy_code      
 left join repo_portfolio rp on r.contract_reference = rp.repo_reference      
 left join currency c2 on rp.ccy_code = c2.ccy_code      
 left join contract_types ct on sub01.contract_type = ct.ct_code_user       
 left join entity e1 on sub01.counterparty_original = e1.entity_code      
 left join security s2 on rp.security_reference = s2.security_reference      
 left join entity e2 on s2.issuer_code = e2.entity_code      
 left join entity_type et2 on e2.et_code_user = et2.entity_type      
 left join contract_guarantee cg on r.contract_reference = cg.benef_contract_ref       
 left join collateral c on cg.guar_contract_ref = c.contract_reference      
 left join currency c3 on c.market_value_ccy = c3.ccy_code       
 left join security s3 on c.security_reference = s3.security_reference       
 left join entity e3 on s3.issuer_code = e3.entity_code      
 left join entity_type et3 on e3.et_code_user = et3.entity_type      
 where sub01.table_name like 'REP%'       
 and sub01.dealbook not in ('HO_B','HO_T','RZBA_B')      
 )       
SELECT *     
FROM combine_repo      
WHERE table_name = 'REPO'""", """select r.partition_key, substr(r.partition_key, 1,8) as reporting_date,book_code as dealbook, r.contract_Type, ct.ct_desc_user,sum(r.cash_nominal*c.exch_Rate) as Cash_Side_EUR	
from repo r	
left join currency c on r.cash_currency = c.ccy_code	
left join contract_types ct on r.contract_Type = ct.ct_code_user	
where r.contract_Type in ('IL0101','IL0201')	
and r.repo_type = 'RR'	
group by r.partition_key, substr(r.partition_key, 1,8) ,book_code , r.contract_Type, ct.ct_desc_user	"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### RUB_CHF_USD

In [13]:
file_name = rd + '_Auswertung_RUB_CHF_USD.xlsx'
sheet_name = [rd + " Data"]
ws_nr = [46]
sqls = ["""
select partition_key, booking_company, sum(ead_pre_ccf) as ead_pre_ccf, sum(ead) as ead, sum(rwa)as RWA, exchange_rate, currency, reporting_date
from Rzb_CDR
where currency in ('USD', 'CHF', 'RUB')
group by partition_key, booking_company, exchange_rate, currency, reporting_date 
"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### RBI_CONS_UNCONS

In [14]:
file_name = 'RBI_cons_uncons.xlsx'
sheet_name = ["Cons (RWAs per Profit Center)", "Uncons (RWAs per Profit Center)", "Cons (inkl.attribute_5)", "Uncons (inkl.attribute_5)"]
ws_nr = [2, 194, 2, 194]
sqls = [
"""
select reporting_Date, partition_key, booking_company, 
case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end as Exposure_type, 
profit_center,  'RBI_GRP' as consolidation_level, bs_indicator, sum(rwa) as rwa
from rzb_Cdr
where record_source = 'T_CDR'
and booking_company = 'RBI'  
group by reporting_Date, partition_key, booking_company, profit_center, bs_indicator, case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end""",
"""
select reporting_Date, partition_key, booking_company, 	
case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end as Exposure_type, 	
profit_center,  'SOLO' as consolidation_level, bs_indicator, sum(rwa) as rwa	
from rzb_Cdr	
where record_source = 'T_CDR'	
and booking_company = 'RBI'  	
group by reporting_Date, partition_key, booking_company, profit_center, bs_indicator, case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end	""",
"""
select reporting_Date, partition_key, booking_company, attribute_5,
case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end as Exposure_type, 
profit_center,  'RBI_GRP' as consolidation_level, bs_indicator, sum(rwa) as rwa
from rzb_Cdr
where record_source = 'T_CDR'
and booking_company = 'RBI'  
group by reporting_Date, partition_key, booking_company,attribute_5, profit_center, bs_indicator, case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end
""",
"""
select reporting_Date, partition_key, booking_company, attribute_5,	
case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end as Exposure_type, 	
profit_center,  'SOLO' as consolidation_level, bs_indicator, sum(rwa) as rwa	
from rzb_Cdr	
where record_source = 'T_CDR'	
and booking_company = 'RBI' 	
group by reporting_Date, partition_key, booking_company,attribute_5, profit_center, bs_indicator, case when table_name = 'EQUITY' then 'Equity exposure' else 'Other exposure' end	
"""]
save_file(file_name, sheet_name, ws_nr, sqls)
modify_cons_uncons(file_name)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.
DataFrame is written to Excel File successfully.


### ECB Daten

In [15]:
file_name = rd + '_ECB_Daten.xlsx'
sheet_name = [rd + "_ECB_Daten"]
ws_nr = [2]
sqls = ["""
WITH	
datatape	
as 	
(	
select r.reporting_Date, 	
r.booking_company as A_booking_entity,	
r.counterparty_name as B_customer_name,	
r.gcc_head_name as C_GCC_name,	
e1.entity_code_2 as D_local_customer,	
r.counterparty_code as E_group_customer_id,	
r.gcc_head_code as F_GCC_id,	
e1.country_code as G_country_of_risk,	
r.industry_code_1 as H_customer_industry,	
r.nace_code as I_Nace_Code,	
r.counterparty_type as J_counterparty_type_number,	
case when r.method_original in ('FOU','STD') then r.method_original	
else 'No Credit Risk*' end 	
as K_basel_approach,	
sum(r.rwa) as L_RWA,	
sum(r.ead_pre_ccf) as M_exposure,	
sum(r.ead) as N_exposure_after_crm,	
sum(r.ead) as O_EAD,	
sum(r.provisions) as P_provision,	
sum(r.ead_pre_ccf) as Q_gross_carrying_amount,	
sum(r.ofb_gua) as R_ofb_gua,	
sum(r.ofb_fac) as S_ofb_undrawn,	
sum(r.ofb_other) as T_ofb_other,	
sum(nvl(r.wcv,0)+nvl(r.wcv_guar,0)) as U_wcv,	
sum(r.wcv_guar) as V_wcv_guar,	
sum(W_wcv_re) as W_wcv_re,	
sum(X_wcv_fin_col) as X_wcv_fin_col,	
sum(Y_wcv_other) as Y_wcv_other,	
sum(ead_pre_ccf_secured) as Z_reg_coll_value,	
sum(AA_reg_coll_gua) as AA_reg_coll_gua,	
sum(AB_reg_coll_re) as AB_reg_coll_re,	
sum(AC_reg_coll_fin) as AC_reg_coll_fin,	
sum(AD_reg_coll_oth) as AD_reg_coll_oth,	
r.internal_rating as AE_most_recent_rating,	
si.rating_Date as AF_rating_date,	
r.rating_model as AG_rating_model,	
case when r.client_risk_status is null then 'S' else r.client_risk_status end as AH_customer_status,	
r.operating_subsegment as AI_responsible_department,	
AJ_forebearance as AJ_forebearance,	
AK_non_performing as AK_non_performing,	
r.default_flag as AL_default_flag,	
AM_actual_dpd as AM_actual_dpd,	
'to populate' as AN_highest_DPD,	
'to populate' as AO_date_last_default,	
r.partition_key,	
r.workout_type,	
r.ead_filter	
from 	
(select r.*,	
case when r.collateral_type in ('RRE','CRE','OTHER_COL')  then r.wcv end as W_wcv_re,	
case when r.collateral_type in ( 'IL0100','IL0200','IL0400','CSH','EQT','GOLD','DSC','MUF','OTCN','RPTN','BSN')  then r.wcv end as X_wcv_fin_col,	
case when r.collateral_type in ('REC','OPC')  then r.wcv end as Y_wcv_other,-- here to ask Rupert if OTHER_COL (collateral type 16) is ok to be included in here	
case when r.collateral_type in ('DRM','GUA','LIN')  then r.ead_pre_ccf end as AA_reg_coll_gua,	
case when r.collateral_type in ('RRE','CRE','OTHER_COL')  then r.ead_pre_ccf end as AB_reg_coll_re,	
case when r.collateral_type in ( 'IL0100','IL0200','IL0400','CSH','EQT','GOLD','DSC','MUF','OTCN','RPTN','BSN')  then r.ead_pre_ccf end as AC_reg_coll_fin,	
case when r.collateral_type in ('REC','OPC')  then r.ead_pre_ccf end as AD_reg_coll_oth,-- here to ask Rupert if Other collateral (non B2) is ok to be included in here	
case when (r.product_group = 'GUARANTEE ISSUED' and r.bs_type = 'OFB') then r.ead_pre_ccf end as ofb_gua,	
case when (r.product_group = 'FACILITY' and r.bs_type = 'OFB') then r.ead_pre_ccf end as ofb_fac,	
case when (r.product_group not in ('FACILITY','GUARANTEE ISSUED') and r.bs_type = 'OFB') then r.ead_pre_ccf end as ofb_other,	
case	
 when (r.default_flag = 'T') then 'Y'	
 when (fb.exposure_classification in ('FBNPD','FBNPDup') and r.default_flag is null) then 'Y'	
 when (fb.exposure_classification in ('FBNPD','FBNPDup') and r.default_flag ='F') then 'Y'	
 when fb.exposure_classification in ('FBNPup', 'FBNP') then 'Y'	
 else 'N'	
end AK_non_performing,	
case    	
 when fb.exposure_classification in ('FBPup', 'FBNPup', 'FBNPDup', 'FBP', 'FBNP', 'FBNPD') then 'Y'	
 else 'N'	
end AJ_forebearance,	
fb.dpd_asof_repdate as AM_actual_dpd,	
case	
when concat(r.booking_company,r.counterparty_code) = 'RBCZ72184' then 'E'	
when concat(r.booking_company,r.counterparty_code) = 'RBCN681989' then 'E'	
when concat(r.booking_company,r.counterparty_code) = 'RBHU136546' then 'E'	
when concat(r.booking_company,r.counterparty_code) = 'RBHU225603' then 'E'	
when concat(r.booking_company,r.counterparty_code) = 'RBHU281128' then 'E'	
when concat(r.booking_company,r.counterparty_code) = 'RBHU432143' then 'E'	
when concat(r.booking_company,r.counterparty_code) = 'RBHU432146' then 'E'	
when concat(r.booking_company,r.counterparty_code) = 'RBHU432147' then 'E'	
when r.crs_local = 'W' then nvl(crs.workout_type,'L') 	
when (r.client_risk_status = 'W' and r.CRS_local != 'W') then 'no differentiation E/L'	
else nvl(crs.workout_type,'n./a.')	
 end workout_type,	
sum(ead) over (partition by r.booking_company, r.counterparty_code) as ead_filter	
from rzb_cdr r	
left join (select attribute_5, exposure_classification, dpd_asof_repdate from rzb_forbearance) fb on fb.attribute_5 = r.attribute_5	
left join rzb_client_risk_status crs 	
on concat(substr(crs.partition_key,0,8),concat(crs.booking_company, crs.counterparty_code)) = concat(substr(r.partition_key,0,8),concat(r.booking_company, r.counterparty_code))	
) r	
left join entity e1 on r.counterparty_code = e1.entity_code	
left join (select entity_code, rating_Date from issuer_credit_ratings where agency_code = 'INTERNAL')si on r.counterparty_code = si.entity_code	
group by r.reporting_Date, 	
r.booking_company,	
r.counterparty_name,	
r.gcc_head_name,	
e1.entity_code_2,	
r.counterparty_code,	
r.gcc_head_code,	
e1.country_code,	
r.industry_code_1,	
r.nace_code,	
r.method_original,	
r.counterparty_type,	
r.internal_rating,	
si.rating_Date,	
r.rating_model,	
r.client_risk_status,	
r.operating_subsegment,	
r.default_flag,	
AJ_forebearance,	
AK_non_performing,	
AM_actual_dpd,	
ead_filter,	
r.crs_local,	
r.workout_type,	
r.partition_key	
order by r.booking_company, r.counterparty_code, r.gcc_head_name, e1.entity_code_2, e1.country_code, r.nace_code, r.method_original	
)	
select 	
d.a_booking_entity,	
d.b_customer_name,	
d.c_gcc_name,	
d.d_local_customer,	
d.e_group_customer_id,	
d.f_gcc_id,	
d.j_counterparty_type_number,	
d.m_exposure,	
d.o_ead,	
g.gcc_ead as AS_GCC_EAD, 	
g.gcc_ead_pre_ccf as AT_GCC_EAD_PRE_CCF,	
d.partition_key,	
d.reporting_date	
from datatape d	
left join (	
        select gcc_head_code, sum(ead_pre_ccf) as gcc_ead_pre_ccf, sum(ead) as gcc_ead	
        from rzb_cdr 	
        group by gcc_head_code	
        ) g 	
on d.F_GCC_id = g.gcc_head_code 	
where (J_counterparty_type_number in ('3','5','4','13','16','39','40','36','62','51','52','28','58','33') 	
    or J_counterparty_type_number in ('18')	
    or J_counterparty_type_number in ('6','41','63'))	
and ead_filter > 10000	
"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### SME Support

In [16]:
file_name = rd + '_SME_SUPPORT.xlsx'
sheet_name = [rd + "_SME_SUPPORT"]
ws_nr = [194]
sqls = ["""
WITH with_esr as (		
	select distinct entity_code, actual_entity_class from esr	
	where original_entity_class <> actual_entity_class	
		and actual_entity_class in ('SME','SME_PF')
)		
, with_rzb_cdr as (		
	select booking_company, counterparty_code, counterparty_new_code, asset_class_original, asset_class_new	
		, bis_entity_type_original, bis_entity_type_new, counterparty_name, counterparty_new_name, counterparty_type
		, counterparty_new_type, counterparty_type_desc, counterparty_new_type_desc, internal_rating, internal_rating_new
		, ead_pre_ccf, rwa
	from rzb_cdr	
	where record_source = 'T_CDR'	
		and rwa > 0
)		
, OUTCOME as (		
	select r.booking_company, r.counterparty_code as counterparty, counterparty_name as entity_desc, r.asset_class_original as asset_class	
		, r.bis_entity_type_original as entity_type, counterparty_type as et_code_user, counterparty_type_desc as entity_type_desc
		, internal_rating as long_term_Rating
		, sum(ead_pre_ccf) exposure, sum(rwa) rwa
	from with_rzb_cdr r	
	where counterparty_code in (select entity_code from with_esr)	
		and internal_rating not like 'CORP%'
		and internal_rating not like 'SME%'
		and internal_rating not like 'SLOT%'
	group by r.booking_company, r.counterparty_code, counterparty_name, r.asset_class_original	
		, r.bis_entity_type_original, counterparty_type, counterparty_type_desc, internal_rating
	UNION	
	select r.booking_company, r.counterparty_new_code as counterparty, counterparty_new_name as entity_desc, r.asset_class_new as asset_class	
		, r.bis_entity_type_new as entity_type, counterparty_new_type as et_code_user, counterparty_new_type_desc as entity_type_desc
		, internal_rating_new as long_term_Rating
		, sum(ead_pre_ccf) exposure, sum(rwa) rwa
	from with_rzb_cdr r	
	where counterparty_new_code in (select entity_code from with_esr)	
		and internal_rating_new not like 'CORP%'
		and internal_rating_new not like 'SME%'
		and internal_rating_new not like 'SLOT%'
	group by r.booking_company, r.counterparty_new_code, counterparty_new_name, r.asset_class_new	
		, r.bis_entity_type_new, counterparty_new_type, counterparty_new_type_desc
		, internal_rating_new
)		
select o.booking_company, o.counterparty, o.entity_desc as counterparty_name, o.et_code_user, o.entity_type_desc, o.long_term_rating, o.exposure, o.rwa,		
	case when (s.actual_entity_class = 'SME' and o.entity_type = 'SME_EU') then 'Firm Size Adjustement and SME support factor applied'	
		when (s.actual_entity_class = 'SME' and o.entity_type <> 'SME_EU') then 'Firm Size Adjustement applied'
		else '**Error**'
	end as treatment	
from OUTCOME o		
left join with_esr s on o.counterparty = s.entity_code		
order by treatment
"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### Asia US Portfolio

In [17]:
file_name = 'Asia_US_portfolio.xlsx'
sheet_name = ["1) Data per Limit Consolidated", "2) Data no limit consolidated", "3) Data Overview", "4) WCV"]
ws_nr = [2, 2, 2, 2]
sqls = ["""
select
sub02.method_original as method,
sub02.partition_key,
to_Date(substr(f.partition_key,1,8),'YYYYMMDD') as reporting_date,
pack_install.get_partition_key_date(f.partition_key) as cut_off_date,
f.contract_reference as facility_id,
f.attribute_5 as facility_unique_id,
db.booking_company,
com.company_name,
sub02.local_customer_number,
sub02.counterparty_original as cocunut,
replace(sub02.kundenname, chr(9), '') as customer_name,
rcs.crs as Client_Risk_Status,
sub02.rating as internal_rating,
e.attribute_6 as GICS_industry_group,
e.attribute_5 as GICS_industry,
e.industry_sector as NACE_industry,
sub02.asset_class_original,
sub03.entity_group as gcc_head,
replace(sub03.entity_desc, chr(9), '') as gcc_name,
f.commited as committed,
f.value_date as start_date,
f.maturity_date,
((f.maturity_date - pack_install.get_partition_key_date(f.partition_key))/365) as residual_maturity,
f.until_further_notice,
f.notice_period,
sub02.unconditionally_cancelable,
f.contract_type,
f.amount as total_in_ccy_original,
f.currency,
(f.amount*c.exch_rate) as total_in_eur,
sub01.nr_of_drawings,
sub01.off_balance_drawn_amount as off_balance_drawn,
sub01.on_balance_drawn_amount as on_balance_drawn,
sub01.secured_part as collateralized,
sub01.guaranteed as guaranteed,
sub02.undrawn_amount as undrawn_amount,
sub02.rwa as rwa_limit,
sub01.rwa_drawn as rwa_drawn
from facility f
left join deal_book db on f.book_code = db.book_code
left join currency c on f.currency = c.ccy_code
left join companies com on db.booking_company = com.company_code
left join
(
select
sub05.partition_key,
sub05.contract_reference,
sub05.method_original,
sub05.counterparty_original,
sub05.asset_class_original,
sub05.unconditionally_cancelable,
sub05.table_name,
sub05.kundenname,
sub05.local_customer_number,
sub05.rating,
sum(sub05.undrawn_amount) as undrawn_amount,
sum(sub05.rwa) as rwa,
sum(sub05.rgc) as rgc
from
(
select
t.partition_key,
t.contract_reference,
t.method_original,
t.counterparty_original,
t.asset_class_original,
case
when t.unconditionnaly_cancelable is null then 'F'
else t.unconditionnaly_cancelable
end unconditionally_cancelable,
t.table_name,
e.entity_desc as kundenname,
e.entity_code_2 as local_customer_number,
t.internal_rating_original as rating,
t.ead_pre_ccf as undrawn_amount,
t.rwa,
t.rgc
from t_cdr t
left join entity e on t.counterparty_original = e.entity_code
where t.table_name = 'FACILITY'
)
sub05
group by
sub05.partition_key,
sub05.method_original,
sub05.contract_reference,
sub05.counterparty_original,
sub05.asset_class_original,
sub05.unconditionally_cancelable,
sub05.table_name,
sub05.kundenname,
sub05.local_customer_number,
sub05.rating
)
sub02 on f.contract_reference = sub02.contract_reference
left join entity e on sub02.counterparty_original = e.entity_code
left join
(
select
sub04.facility_reference,
count(distinct sub04.contract_reference) as nr_of_drawings,
sum(sub04.ofb) as off_balance_drawn_amount,
sum(sub04.onb) as on_balance_drawn_amount,
sum(sub04.secured_part) as secured_part,
sum(sub04.guaranteed) as guaranteed,
sum(sub04.rwa_drawn) as rwa_drawn
from
(
select
facility_reference,
contract_reference,
case
when bs_type = 'OFB' then ead_pre_ccf
else 0
end ofb,
case
when bs_type <> 'OFB' then ead_pre_ccf
else 0
end onb,
case
when tranche_collateral_type = 'COLLATERALIZED' then ead_pre_ccf
else 0
end secured_part,
case
when tranche_guarantee_type = 'GUARANTEED'
then ead_pre_ccf
else 0
end guaranteed,
rwa as rwa_drawn
from t_cdr
where facility_reference is not null
)
sub04
group by
sub04.facility_reference
)
sub01 on f.contract_reference = sub01.facility_reference
left join 
(
select
e.entity_group,
e.entity_code,
ent.entity_desc
from esr e
left join entity ent on (e.entity_group = ent.entity_code)
)
sub03 on (sub02.counterparty_original = sub03.entity_code)
left join 
(select  counterparty_code as ctpy_code,
case
        when client_risk_status is null
        then crs_local
        else client_risk_status
        end crs
from rzb_cdr
group by counterparty_code,
case
        when client_risk_status is null
        then crs_local
        else client_risk_status
        end
)rcs on sub02.counterparty_original  = rcs.ctpy_code
where db.booking_company in ('RBSG','RBMY','RBUS','RBHK','RFHK','RBCN','RBCN2')
""",
"""
select r.partition_key, 
r.booking_company,
r.booking_company_desc,
r.attribute_5 as product_id,
r.bs_type,
r.contract_reference,
r.counterparty_code as borrower_cocunut,
r.counterparty_name as borrower_name,
case
        when r.client_risk_status is null
        then r.crs_local
        else r.client_risk_status
        end as Client_Risk_Status,
r.counterparty_new_code as guarantor_cocunut, 
r.counterparty_new_name as guarantor_name,
r.industry_code_1,
r.industry_code_2,
r.residence_country,
r.risk_country,
r.counterparty_type,
r.counterparty_type_desc,
r.internal_rating,
r.gcc_head_code,
r.gcc_head_name,
r.contract_type,
r.contract_type_desc,
r.currency,
r.start_date,
r.maturity_date,
r.tenor_residual,
r.collateral_type,
r.facility_reference as limit_reference,
sub01.total_limit_amount,
sum(r.ead_pre_ccf) as ead_pre_ccf,
sum(r.ead_pre_ccf_secured) as biii_collateralised,
sum(r.ead_pre_ccf_unsecured) as biii_uncollateralised,
sum(r.rwa) as rwa
from rzb_cdr r
left join 
    (select f.contract_reference, 
    f.amount*c.exch_Rate as total_limit_amount
    from facility f
    left join currency c on f.currency = c.ccy_code)sub01 on r.contract_reference = sub01.contract_Reference
    where booking_company in ('RBSG','RBMY','RBUS','RBCN','RBCN2','RBHK','RFHK')
    and record_source <> 'WCV'
    and table_name <> 'FACILITY'
    and r.facility_reference is null
group by
r.partition_key,
r.booking_company,
r.booking_company_desc,
r.attribute_5,
r.bs_type,
r.contract_reference,
r.counterparty_code,
r.counterparty_name,
r.counterparty_new_code, 
r.counterparty_new_name,
r.industry_code_1,
r.industry_code_2,
r.residence_country,
r.risk_country,
r.counterparty_type,
r.counterparty_type_desc,
r.internal_rating,
r.gcc_head_code,
r.gcc_head_name,
r.contract_type,
r.contract_type_desc,
r.currency,
r.facility_reference,
r.start_date,
r.maturity_date,
r.tenor_residual,
r.collateral_type,
sub01.total_limit_amount,
case
        when r.client_risk_status is null
        then r.crs_local
        else r.client_risk_status
        end
""",
"""
select r.reporting_Date,r.booking_company, r.product_group, r.counterparty_code, 
case when r.counterparty_name like '%;' then substr(r.counterparty_name, 1,LENGTH(r.counterparty_name)-1)
else r.counterparty_name
end as counterparty_name_,
case
        when r.client_risk_status is null
        then r.crs_local
        else r.client_risk_status
        end as Client_rs,
r.internal_rating, 
r.gcc_head_code, 
case when r.gcc_head_name like '%;' then substr(r.gcc_head_name, 1,LENGTH(r.gcc_head_name)-1)
else r.gcc_head_name
end as gcc_head_name_,
residual_maturity, tenor_residual,
sum(r.ead_pre_ccf) as ead_pre_ccf, sum(r.ead) as ead, sum(r.rwa) as rwa
from rzb_cdr r
where r.record_source <> 'WCV'
and r.booking_company in ('RBSG','RBMY','RBUS','RBCN','RBCN2','RBHK','RFHK')
group by r.reporting_Date, r.product_group, r.counterparty_code, 
case
        when r.client_risk_status is null
        then r.crs_local
        else r.client_risk_status
        end,
r.internal_rating, r.gcc_head_code, residual_maturity, tenor_Residual, r.booking_company,
case when r.counterparty_name like '%;' then substr(r.counterparty_name, 1,LENGTH(r.counterparty_name)-1)
else r.counterparty_name
end,
case when r.gcc_head_name like '%;' then substr(r.gcc_head_name, 1,LENGTH(r.gcc_head_name)-1)
else r.gcc_head_name
end
""",
"""
select distinct partition_key, reporting_date, booking_company, counterparty_code, counterparty_name, collateral_type,
sum (nvl (wcv,wcv_guar)) as wcv
from rzb_cdr
where booking_company in ('RBUS', 'RBCN', 'RBCN2', 'RBHK', 'RFHK', 'RBSG', 'RBMY')
group by partition_key, booking_company, counterparty_code, counterparty_name, collateral_type, reporting_date
"""]
save_file(file_name, sheet_name, ws_nr, sqls)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  


DataFrame is written to Excel File successfully.


### CVA

In [18]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/004 Redeployment/CVA"
os.chdir(directory)

file_name = 'CVA.xlsx'
sheet_name = ["CVA"]
ws_nr = [2]
sqls = ["""
select t_cva.* , concat(concat(booking_company,'_CVA_'),substr(partition_key,5,4)) as ids	
from t_cva	
order by booking_company asc , counterparty asc , cva_capital_charge desc 	
"""]
save_file_cva(file_name, sheet_name, ws_nr, sqls)

DataFrame is written to Excel File successfully.


### Redeployment

In [19]:
loc = r"C:\Users\WZHARBC\Documents\Python Monthly Tasks\Exports\004 Redeployment"
    


### Email function

In [20]:
def send_email(to, cc, sub, body, file_name):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to
    mail.Cc = cc
    mail.Subject = sub
    mail.Body = body
    # To attach a file to the email (optional):
    attachment  = file_name
    mail.Attachments.Add(attachment)

    mail.Display()

In [21]:
contacts = {
    'RUB, CHF, USD Exposure': {
        'to': 'christoph.ilgner@rbinternational.com; martin.ulman@rbinternational.com',
        'cc': '',
        'file' : rd + '_Auswertung_RUB_CHF_USD.xlsx'
    }
    ,
    'ECB-Datenfile_Redeployment': {
        'to': 'klaus.bertha@rbinternational.com',
        'cc': '',
        'file' : rd + '_ECB_Daten.xlsx'
    },
    'OTC-Netting_Redeployment' : {
        'to': 'dorothea.wanek@rbinternational.com',
        'cc': '',
        'file': rd + '_OTC_NETTING.xlsx'
    },
    'RBIAG_Redeployment' : {
        'to': 'guenter.plachy@rbinternational.com',
        'cc': '',
        'file': rd + '_RBIAG.xlsx'
    },
    'RBIAG-Global-Treasury_Redeployment' : {
        'to': 'franz.moldaschl@rbinternational.com',
        'cc': '',
        'file': rd + '_RBIAG_GLOBAL_TREASURY.xlsx'
    },
    'RBSG_Redeployment' : {
        'to': 'carol.ng@sg.rbinternational.com',
        'cc': '',
        'file': rd + '_RBSG.xlsx'
    },
    'REPO_Redeployment' : {
        'to': 'tibor.jakubik@rbinternational.com; eugen.visan@rbinternational.com; ana-maria.kramer@rbinternational.com; vitalii.patenko@rbinternational.com; raluca.primetzhofer@rbinternational.com',
        'cc': 'elena.filipidescu@rbinternational.com; annuradha.tandon@rbinternational.com; maxim.alexandrov@rbinternational.com',
        'file': rd + '_REPO.xlsx'
    },
    'SME-Factor' : {
        'to': 'annuradha.tandon@rbinternational.com',
        'cc': '',
        'file': rd + '_SME_SUPPORT.xlsx'
    },
    'Neue_Valuetypes' : {
        'to': 'guenter.plachy@rbinternational.com; inesa.salkica@rbinternational.com; markus.gruber@rbinternational.com; philipp.gunesch@rbinternational.com; yllka.kadrijaj@rbinternational.com',
        'cc': '',
        'file': 'RBI_cons_uncons.xlsx'
    },
    'CVA ALEASS_Redeployment' : {
        'to' : 'wilhelm.karer@rl.co.at; stanka.lochmann@rl.co.at; christoph.angeli@rl.co.at',
        'cc': '',
        'file' : 'CVA/ALEASS_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA AVAL_Redeployment' : {
        'to' : 'stanislava.dzhygurda@aval.ua',
        'cc': '',
        'file' : 'CVA/AVAL_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA CENTRO_Redeployment' : {
        'to' : 'andreas.adam@rcb.at',
        'cc': '',
        'file' : 'CVA/CENTRO_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA RBCZ_Redeployment' : {
        'to' : 'tomas.rydl@rl.cz; pavel.pesek@rl.cz; pavel.vachalec@rl.cz; matus.jambor@rl.cz',
        'cc': '',
        'file' : 'CVA/RBCZ_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA RBHR_Redeployment' : {
        'to' : 'jasmina.skrbin@rba.hr;',
        'cc': '',
        'file' : 'CVA/RBHR_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA RBHU_Redeployment' : {
        'to' : 'tamas.racz@raiffeisen.hu;',
        'cc': '',
        'file' : 'CVA/RBHU_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA RBI_Redeployment' : {
        'to' : 'markus.gruber@rbinternational.com; guenter.plachy@rbinternational.com;',
        'cc': '',
        'file' : 'CVA/RBI_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA RBRU_Redeployment' : {
        'to' : 'RBRU-RWA_Calculation@raiffeisen.ru;Ekaterina.GARMAEVA@raiffeisen.ru;Ruslan.GUBIEV@raiffeisen.ru',
        'cc': '',
        'file' : 'CVA/RBRU_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA RLBANK_Redeployment' : {
        'to' : 'wilhelm.karer@rl.co.at; stanka.lochmann@rl.co.at; christoph.angeli@rl.co.at',
        'cc': '',
        'file' : 'CVA/RLBANK_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA RLCZ_Redeployment' : {
        'to' : 'tomas.rydl@rl.cz; pavel.pesek@rl.cz; pavel.vachalec@rl.cz; matus.jambor@rl.cz',
        'cc': '',
        'file' : 'CVA/RLCZ_CVA_' + rd[-4:] + '.xlsx'
    },
    'CVA TBSK_Redeployment' : {
        'to' : 'ivana_popovicova@tatrabanka.sk',
        'cc': '',
        'file' : 'CVA/TBSK_CVA_' + rd[-4:] + '.xlsx'
    }
}


### Send emails

In [22]:
body = """Dear colleagues, 
please find attached current RAY data. 
BR 
CPC 
"""

directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/004 Redeployment"
for contact in contacts:
    send_email(contacts[contact]['to'], contacts[contact]['cc'], contact, body, directory + "/" + contacts[contact]['file'])

## You have to do another file as well.

https://wiki.rbinternational.corp/confluence/pages/viewpage.action?pageId=460301294

# Send contactperson

In [ ]:
contacts_nwu = {
    'TBSK': {
        'to': 'ivana_popovicova@tatrabanka.sk;michaela_rybova@tatrabanka.sk',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'TLSK': {
        'to': 'jozef_rak@tatrabanka.sk',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBBH': {
        'to': 'selma.ibisevic-dinar@raiffeisengroup.ba; lejla.niksic@raiffeisengroup.ba',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLBH': {
        'to': 'edita.cagalj@raiffeisengroup.ba; amila.dacic@raiffeisengroup.ba',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBBY': {
        'to': 'Elena.Rayman@priorbank.by;Yuliya.Bahvalova@priorbank.by;Olga.Laschevskaya@priorbank.by;natalya.sapezinskaya@priorbank.by;Dmitry.Kovalchuk@priorbank.by',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLBY1': {
        'to': 'natalya.truhan@priorbank.by; natalya.vusheva@priorbank.by',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com; anna.baranova@priorbank.by',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'AVAL': {
        'to': 'stanislava.dzhygurda@aval.ua',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RBAL': {
        'to': 'Lira.Veshi@raiffeisen.al;jonida.shirokaj@raiffeisen.al',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RLAL': {
        'to': 'holta.balli@raiffeisen.al; anida.nako@raiffeisen.al',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RBRU': {
        'to': 'RBRU-RWA_Calculation@raiffeisen.ru;Ekaterina.PURTOVA@raiffeisen.ru;Nadezhda.MAZAEVA@raiffeisen.ru;Vasily.LUKYANOV@raiffeisen.ru',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RBRUFIN': {
        'to': 'Valentina.AYUPOVA@raiffeisen.ru;Maksim.SAPOZHNIKOV@raiffeisen.ru',
        'cc': 'nertila.gjakova@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Christoph Gamauf'
    },
    'RBRS': {
        'to': 'srdjan.milosevic@raiffeisenbank.rs;anad.nikolic@raiffeisenbank.rs',
        'cc': 'nertila.gjakova@rbinternational.com; martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Martin Berger'
    },
    'RLRS': {
        'to': 'miodrag.milovanovic@raiffeisen-rent.rs;ivana.boricic@raiffeisen-leasing.rs;maja.isakovic@raiffeisen-leasing.rs;tatjana.marjanov@raiffeisen-leasing.rs',
        'cc': 'nertila.gjakova@rbinternational.com; martin.berger@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Martin Berger'
    },
    'RBKO': {
        'to': 'muhamet.aliu@raiffeisen-kosovo.com; ilir.kallaba@raiffeisen-kosovo.com; etleve.grajcevci@raiffeisen-kosovo.com',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RLKO': {
        'to': 'fatos.karacica@raiffeisen-kosovo.com;fatmir.uka@raiffeisen-kosovo.com',
        'cc': 'nertila.gjakova@rbinternational.com; flori.luci@rbinternational.com',
        'coll1' : 'Nertila Gjakova',
        'coll2' : 'Flori Luci'
    },
    'RFACTOR': {
        'to': 'bernd.kaiser@raiffeisen-factorbank.at;christoph.igler@raiffeisen-factorbank.at;thomas.fuehringer@raiffeisen-factorbank.at',
        'cc': 'markus.hochleitner@rbinternational.com; christoph.gamauf@rbinternational.com',
        'coll1' : 'Markus Hochleitner',
        'coll2' : 'Christoph Gamauf'
    }
}

In [ ]:
def send_email2(to, cc, sub, coll1, coll2):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = to
    mail.Cc = cc
    mail.Subject = sub + " - Contact for this month"
    mail.Body = """Dear colleagues,

Since """+ coll1 +""" is going to be on vacation for some time, i want to inform you that if you have question regarding this monthly process, your spokeperson for this month is """+ coll2 +""".\nSo if you have questions, please contact """+coll2+"""

Kind regards,
Arber Bajraktari
"""

    mail.Display()

In [ ]:
for contact in contacts_nwu:
    send_email2(contacts_nwu[contact]['to'], contacts_nwu[contact]['cc'], contact, contacts_nwu[contact]['coll1'], contacts_nwu[contact]['coll2'])